# Multi-Agent Collaboration with Microsoft Agent Framework
---

## What is Multi-Agent Collaboration?
Multi-Agent Collaboration refers to the process where multiple autonomous agents—each capable of independent decision-making—work together to achieve common or complementary objectives. This concept is widely used in fields like artificial intelligence, robotics, distributed computing, and simulation, and it involves several key aspects:

- **Effective Communication and Coordination**:
Agents exchange information and align their actions to collectively achieve a goal, ensuring that tasks are organized and synchronized.

- **Autonomous, Distributed Decision-Making**:
Each agent operates independently, making local decisions while contributing to a broader strategy, which enhances flexibility and fault tolerance.

- **Adaptive Task Specialization**:
Agents focus on specific roles or subtasks based on their capabilities, and they adjust their strategies through iterative feedback, leading to improved overall performance.

## Key Advantages
- **Efficiency Through Task Specialization**:
By assigning specific roles to different agents, the system can handle complex tasks in parallel. This specialization allows each agent to focus on its area of expertise, resulting in faster and more effective problem-solving.

- **Scalability and Flexibility**:
Structured communication and dynamic task allocation enable the system to scale easily. It can adapt to varying project complexities by adding or reassigning agents as needed, ensuring that the collaboration remains robust even as demands change.

- **Enhanced Iterative Refinement**:
Built-in feedback loops and iterative dialogue facilitate continuous improvement. Agents can refine their outputs based on real-time feedback, leading to more accurate and cohesive final results.

## Three Multi-Agent Patterns in Microsoft Agent Framework

This notebook demonstrates three powerful multi-agent collaboration patterns:

| Pattern | Description | Use Case |
|---------|-------------|----------|
| **Group Chat** | Sequential conversation with turn-based interaction | Collaborative refinement, approval workflows |
| **Concurrent Execution** | Parallel execution with fan-out/fan-in | Independent parallel analysis, diverse perspectives |
| **Magentic Orchestration** | Intelligent orchestration with planning | Complex multi-step tasks, research + execution |

**References:**
- [Microsoft Agent Framework Documentation](https://learn.microsoft.com/en-us/agent-framework/)
- [AutoGen paper: Enabling Next-Gen LLM Applications via Multi-Agent Conversation](https://arxiv.org/abs/2308.08155)
- [Multi-Agent Collaboration Concepts](https://langchain-ai.github.io/langgraph/concepts/multi_agent/#network)

# Setup
---
Install required packages and import libraries for Microsoft Agent Framework.

In [4]:
import asyncio
import os
from typing import Annotated, Any
from dotenv import load_dotenv

# Microsoft Agent Framework core
from agent_framework import (
    ChatAgent, 
    ChatMessage, 
    TextContent, 
    Role,
    ConcurrentBuilder,
    MagenticBuilder,
    MagenticCallbackEvent,
    MagenticOrchestratorMessageEvent,
    MagenticAgentDeltaEvent,
    MagenticAgentMessageEvent,
    MagenticFinalResultEvent,
    MagenticCallbackMode,
    WorkflowOutputEvent,
    HostedCodeInterpreterTool
)

# Azure integrations
from agent_framework.azure import AzureOpenAIChatClient, AzureAIAgentClient
from agent_framework.openai import OpenAIChatClient, OpenAIResponsesClient
from azure.identity.aio import AzureCliCredential

# Tool decorators
from agent_framework import ai_function

load_dotenv(override=True)
print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## Environment Configuration
---
Configure Azure OpenAI and Azure AI Project settings.

In [5]:
# Check required environment variables
print("Checking environment variables...")

# For Azure OpenAI
azure_openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_openai_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
azure_openai_version = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")

# For Azure AI Foundry
azure_project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")

if azure_openai_endpoint and azure_openai_deployment:
    print("✓ Azure OpenAI configuration found")
    print(f"  - Endpoint: {azure_openai_endpoint}")
    print(f"  - Deployment: {azure_openai_deployment}")
else:
    print("❌ Missing required environment variables!")
    print("\nRequired environment variables:")
    print("  - AZURE_OPENAI_ENDPOINT")
    print("  - AZURE_OPENAI_API_KEY")
    print("  - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

# Create shared chat client for reuse
azure_chat_client = AzureOpenAIChatClient(
    model_id=azure_openai_deployment,
    endpoint=azure_openai_endpoint,
    api_key=azure_openai_key,
    api_version=azure_openai_version
)

print("\n✓ Chat client created successfully")

Checking environment variables...
✓ Azure OpenAI configuration found
  - Endpoint: https://hyo-ai-foundry-pjt1-resource.openai.azure.com/
  - Deployment: gpt-4.1-mini

✓ Chat client created successfully


# Case 1: Group Chat Pattern
---

## What is Group Chat?
Group Chat is a sequential conversation pattern where agents take turns communicating. It's ideal for:
- **Collaborative refinement**: Multiple rounds of review and improvement
- **Approval workflows**: Task completion with validation steps
- **Turn-based dialogue**: Natural conversation flow with clear speaker transitions

## Architecture

```
User Input → Agent 1 (Generate) → Agent 2 (Review) → Agent 1 (Refine) → ... → Termination
```

### Key Features:
- ✅ Sequential turn-taking
- ✅ Custom termination strategies
- ✅ Context-aware conversation history
- ✅ Role-based specialization

## Use Case: Copywriting with Art Director Review
We'll create a two-agent system where:
1. **CopyWriter Agent**: Generates creative slogans
2. **ArtDirector Agent**: Reviews and provides feedback until approval

## 🧪 Step 1.1: Define Custom Termination Strategy
---
Create a termination strategy that stops when the art director approves the copy.

In [6]:
class ApprovalTerminationStrategy:
    """Custom termination strategy that stops when approval is given."""
    
    def __init__(self, max_iterations: int = 10):
        self.max_iterations = max_iterations
        self.iteration_count = 0
    
    def should_terminate(self, messages: list[ChatMessage]) -> bool:
        """Check if the conversation should terminate."""
        self.iteration_count += 1
        
        # Terminate if max iterations reached
        if self.iteration_count >= self.max_iterations:
            print(f"\n⚠️ Reached maximum iterations ({self.max_iterations})")
            return True
        
        # Terminate if last message contains approval
        if messages and len(messages) > 0:
            last_message = messages[-1]
            if last_message.text and "approved" in last_message.text.lower():
                print(f"\n✅ Approval detected in iteration {self.iteration_count}")
                return True
        
        return False
    
    def reset(self):
        """Reset the iteration counter."""
        self.iteration_count = 0

print("✓ Termination strategy defined")

✓ Termination strategy defined


## 🧪 Step 1.2: Create Specialized Agents
---
Define the CopyWriter and ArtDirector agents with specific roles and instructions.

In [7]:
# Agent definitions
COPYWRITER_NAME = "CopyWriter"
COPYWRITER_INSTRUCTIONS = """
You are a copywriter with ten years of experience and are known for brevity and a dry humor.
The goal is to refine and decide on the single best copy as an expert in the field.
Only provide a single proposal per response.
You're laser focused on the goal at hand.
Don't waste time with chit chat.
Consider suggestions when refining an idea.
"""

ARTDIRECTOR_NAME = "ArtDirector"
ARTDIRECTOR_INSTRUCTIONS = """
You are an art director who has opinions about copywriting born of a love for David Ogilvy.
The goal is to determine if the given copy is acceptable to print.
If so, state that it is approved. Do not use the word "approve" unless you are giving approval.
If not, provide insight on how to refine suggested copy without example.
"""

# Create agents
copywriter_agent = ChatAgent(
    chat_client=azure_chat_client,
    name=COPYWRITER_NAME,
    instructions=COPYWRITER_INSTRUCTIONS
)

artdirector_agent = ChatAgent(
    chat_client=azure_chat_client,
    name=ARTDIRECTOR_NAME,
    instructions=ARTDIRECTOR_INSTRUCTIONS
)

print("✓ Agents created:")
print(f"  - {COPYWRITER_NAME}: Creative slogan generator")
print(f"  - {ARTDIRECTOR_NAME}: Quality reviewer and approver")

✓ Agents created:
  - CopyWriter: Creative slogan generator
  - ArtDirector: Quality reviewer and approver


## 🧪 Step 1.3: Execute Group Chat
---
Run the group chat with turn-based conversation until approval or max iterations.

In [8]:
async def run_group_chat():
    """Execute a group chat between copywriter and art director."""
    
    # Task to accomplish
    TASK = "Create a slogan for a new line of electric cars."
    
    print("=" * 80)
    print("CASE 1: GROUP CHAT PATTERN - Copywriting Review")
    print("=" * 80)
    print(f"\n📋 Task: {TASK}\n")
    
    # Initialize termination strategy
    termination = ApprovalTerminationStrategy(max_iterations=10)
    
    # Create conversation history
    messages: list[ChatMessage] = [
        ChatMessage(role=Role.USER, contents=[TextContent(text=TASK)])
    ]
    
    # Track current speaker
    current_speaker = copywriter_agent
    iteration = 0
    
    print(f"👤 User: {TASK}\n")
    print("-" * 80 + "\n")
    
    # Main conversation loop
    while not termination.should_terminate(messages):
        iteration += 1
        
        # Get agent response
        response = await current_speaker.run(messages[-1].text if messages else TASK)
        
        # Display response
        print(f"🤖 {current_speaker.name} (Iteration {iteration}):")
        print(f"{response.text}\n")
        print("-" * 80 + "\n")
        
        # Add response to history
        messages.append(ChatMessage(
            role=Role.ASSISTANT,
            contents=[TextContent(text=response.text)],
            author_name=current_speaker.name
        ))
        
        # Switch speaker
        current_speaker = artdirector_agent if current_speaker == copywriter_agent else copywriter_agent
    
    print("\n" + "=" * 80)
    print("GROUP CHAT COMPLETED")
    print("=" * 80)
    print(f"\nTotal iterations: {iteration}")
    print(f"Final slogan approved: {messages[-1].text}")

# Run the group chat
await run_group_chat()

CASE 1: GROUP CHAT PATTERN - Copywriting Review

📋 Task: Create a slogan for a new line of electric cars.

👤 User: Create a slogan for a new line of electric cars.

--------------------------------------------------------------------------------



🤖 CopyWriter (Iteration 1):
Silent power. Zero excuses.

--------------------------------------------------------------------------------

🤖 ArtDirector (Iteration 2):
The copy has an impactful, terse style that conveys strength and determination. However, it leans heavily on vague abstraction. To sharpen it, ensure the message connects to a tangible benefit or audience insight. Ground the emotions in a specific context or product attribute to make it resonate more deeply. Avoid generic, feel-good phrases that float without a clear anchor.

--------------------------------------------------------------------------------

🤖 CopyWriter (Iteration 3):
Under pressure, you break records—not promises. Rugged gear that stands up when you don’t.

--------------------------------------------------------------------------------

🤖 ArtDirector (Iteration 4):
This copy has a strong, clear message and an engaging contrast between breaking records and keeping promises. The phrasing is punchy and spe

### 📊 Group Chat Pattern Summary

**Advantages:**
- ✅ Clear turn-based communication
- ✅ Easy to trace conversation flow
- ✅ Natural for collaborative refinement
- ✅ Simple to implement termination logic

**Considerations:**
- ⚠️ Sequential execution (not parallel)
- ⚠️ Potential for circular discussions
- ⚠️ Requires careful termination strategy

**Best Use Cases:**
- Document review and approval
- Iterative content refinement
- Quality assurance workflows
- Collaborative decision-making

# Case 2: Concurrent Execution Pattern
---

## What is Concurrent Execution?
Concurrent execution is a fan-out/fan-in pattern where multiple agents work in parallel. It's ideal for:
- **Independent analysis**: Each agent provides unique perspective
- **Parallel processing**: Faster completion through simultaneous work
- **Diverse insights**: Multiple viewpoints on the same problem

## Architecture

```
                    ┌──→ Researcher Agent ──┐
User Input ─────────┼──→ Marketer Agent ───┼───→ Aggregated Results
                    └──→ Legal Agent ───────┘
```

### Key Features:
- ✅ Parallel execution (fan-out)
- ✅ Automatic aggregation (fan-in)
- ✅ Independent agent perspectives
- ✅ Fast completion

## Use Case: Multi-Perspective Product Analysis
We'll create three agents that analyze a product launch simultaneously:
1. **Researcher**: Market insights and analysis
2. **Marketer**: Marketing strategy and messaging
3. **Legal**: Compliance and risk assessment

## 🧪 Step 2.1: Create Domain-Specific Agents
---
Define specialized agents for research, marketing, and legal review.

In [9]:
# Create domain-specific agents
researcher_agent = azure_chat_client.create_agent(
    name="Researcher",
    instructions="""
    You are an expert market and product researcher. Given a prompt, provide concise, 
    factual insights, opportunities, and risks. Focus on data-driven analysis.
    """
)

marketer_agent = azure_chat_client.create_agent(
    name="Marketer",
    instructions="""
    You are a creative marketing strategist. Craft compelling value propositions 
    and target messaging aligned to the prompt. Be creative and customer-focused.
    """
)

legal_agent = azure_chat_client.create_agent(
    name="Legal",
    instructions="""
    You are a cautious legal/compliance reviewer. Highlight constraints, disclaimers, 
    and policy concerns based on the prompt. Focus on risk mitigation.
    """
)

print("✓ Domain agents created:")
print("  - Researcher: Market analysis expert")
print("  - Marketer: Creative strategist")
print("  - Legal: Compliance reviewer")

✓ Domain agents created:
  - Researcher: Market analysis expert
  - Marketer: Creative strategist
  - Legal: Compliance reviewer


## 🧪 Step 2.2: Build Concurrent Workflow
---
Use ConcurrentBuilder to create a fan-out/fan-in workflow.

In [10]:
async def run_concurrent_workflow():
    """Execute concurrent workflow with multiple agents."""
    
    # Task to accomplish
    TASK = "We are launching a new budget-friendly electric bike for urban commuters."
    
    print("=" * 80)
    print("CASE 2: CONCURRENT EXECUTION PATTERN - Multi-Perspective Analysis")
    print("=" * 80)
    print(f"\n📋 Task: {TASK}\n")
    print("🔄 Executing agents in parallel...\n")
    
    # Build concurrent workflow
    workflow = (
        ConcurrentBuilder()
        .participants([researcher_agent, marketer_agent, legal_agent])
        .build()
    )
    
    # Run workflow
    events = await workflow.run(TASK)
    outputs = events.get_outputs()
    
    print("=" * 80)
    print("CONCURRENT EXECUTION COMPLETED")
    print("=" * 80)
    
    # Display aggregated results
    if outputs:
        print("\n📊 Aggregated Conversation (All Agent Responses):\n")
        for output in outputs:
            messages: list[ChatMessage] | Any = output
            for i, msg in enumerate(messages, start=1):
                name = msg.author_name if msg.author_name else "User"
                print(f"\n{'─' * 80}")
                print(f"[{i:02d}] {name}:")
                print(f"{'─' * 80}")
                print(f"{msg.text}\n")
    else:
        print("\n⚠️ No outputs received from workflow")

# Run concurrent workflow
await run_concurrent_workflow()

CASE 2: CONCURRENT EXECUTION PATTERN - Multi-Perspective Analysis

📋 Task: We are launching a new budget-friendly electric bike for urban commuters.

🔄 Executing agents in parallel...



CONCURRENT EXECUTION COMPLETED

📊 Aggregated Conversation (All Agent Responses):


────────────────────────────────────────────────────────────────────────────────
[01] User:
────────────────────────────────────────────────────────────────────────────────
We are launching a new budget-friendly electric bike for urban commuters.


────────────────────────────────────────────────────────────────────────────────
[02] Researcher:
────────────────────────────────────────────────────────────────────────────────
Insights:
- Market Demand: Urban electric bike sales are growing at a CAGR of ~12-15%, driven by increasing urban congestion and environmental concerns.
- Price Sensitivity: Budget-friendly segments (<$1000) attract a large portion of first-time e-bike buyers and cost-conscious commuters.
- Features Expectation: Key features include sufficient battery range (20-40 miles), lightweight frame, reliable motor (250-350W), and basic smart connectivity.
- Competition: Major competitors in bu

### 📊 Concurrent Execution Pattern Summary

**Advantages:**
- ✅ Fast parallel execution
- ✅ Multiple independent perspectives
- ✅ Efficient resource utilization
- ✅ Scalable to many agents

**Considerations:**
- ⚠️ No inter-agent communication during execution
- ⚠️ Requires aggregation strategy
- ⚠️ May produce redundant information

**Best Use Cases:**
- Multi-perspective analysis
- Independent task execution
- Rapid information gathering
- Parallel research tasks

# Case 3: Magentic Orchestration Pattern
---

## What is Magentic Orchestration?
Magentic is an intelligent orchestration pattern that uses a manager agent to coordinate specialized agents. It's ideal for:
- **Complex multi-step tasks**: Planning and execution coordination
- **Dynamic agent selection**: Manager chooses the right agent for each step
- **Adaptive workflows**: Adjusts based on progress and results

## Architecture

```
                    ┌──→ Plan → Select Agent → Execute ──┐
User Task ──→ Manager │                                    │──→ Final Result
                    └──→ Monitor Progress → Adjust ──────┘
                         ↓
                    [Researcher, Coder, Analyst...]
```

### Key Features:
- ✅ Intelligent planning and orchestration
- ✅ Dynamic agent selection
- ✅ Progress monitoring
- ✅ Adaptive execution

## Use Case: Complex Research with Code Analysis
We'll create a Magentic workflow with:
1. **Manager Agent**: Orchestrates and plans
2. **Researcher Agent**: Finds information (with web search)
3. **Coder Agent**: Performs computational analysis

## 🧪 Step 3.1: Create Specialized Agents for Magentic
---
Create agents with specific capabilities for research and coding.

In [11]:
# Create researcher agent with search capability
magentic_researcher = ChatAgent(
    name="ResearcherAgent",
    description="Specialist in research and information gathering",
    instructions="""
    You are a Researcher. You find information without additional computation 
    or quantitative analysis. Provide factual, well-researched information.
    """,
    chat_client=azure_chat_client
)

# Create coder agent with code interpreter
magentic_coder = ChatAgent(
    name="CoderAgent",
    description="A helpful assistant that evaluates, writes and executes code to process and analyze data.",
    instructions="""
    You solve questions using code. Please provide detailed analysis and 
    computation process. Always explain your approach and results clearly.
    """,
    chat_client=azure_chat_client,
    tools=HostedCodeInterpreterTool()
)

print("✓ Magentic agents created:")
print("  - ResearcherAgent: Information gathering with search")
print("  - CoderAgent: Computational analysis with code execution")

✓ Magentic agents created:
  - ResearcherAgent: Information gathering with search
  - CoderAgent: Computational analysis with code execution


## 🧪 Step 3.2: Setup Streaming Callbacks
---
Define callback functions to monitor Magentic workflow progress in real-time.

In [12]:
# State for tracking streaming
last_stream_agent_id: str | None = None
stream_line_open: bool = False

async def on_magentic_event(event: MagenticCallbackEvent) -> None:
    """
    Callback to process events emitted by the Magentic workflow.
    Handles orchestrator messages, agent deltas, agent messages, and final results.
    """
    global last_stream_agent_id, stream_line_open
    
    if isinstance(event, MagenticOrchestratorMessageEvent):
        # Display orchestrator planning and coordination messages
        print(f"\n🎯 [ORCHESTRATOR: {event.kind}]")
        print(f"{'─' * 60}")
        print(f"{getattr(event.message, 'text', '')}")
        print(f"{'─' * 60}\n")
        
    elif isinstance(event, MagenticAgentDeltaEvent):
        # Stream agent responses token by token
        if last_stream_agent_id != event.agent_id or not stream_line_open:
            if stream_line_open:
                print()
            print(f"\n🤖 [{event.agent_id}]: ", end="", flush=True)
            last_stream_agent_id = event.agent_id
            stream_line_open = True
        print(event.text, end="", flush=True)
        
    elif isinstance(event, MagenticAgentMessageEvent):
        # Display complete agent messages
        if stream_line_open:
            print(" (complete)")
            stream_line_open = False
            print()
        if event.message is not None:
            response_text = (event.message.text or "").replace("\n", " ")[:200]
            print(f"\n✅ [AGENT: {event.agent_id}] {event.message.role.value}")
            print(f"   {response_text}...")
            print(f"{'─' * 60}\n")
            
    elif isinstance(event, MagenticFinalResultEvent):
        # Display final orchestration result
        print("\n" + "=" * 80)
        print("🎉 MAGENTIC ORCHESTRATION - FINAL RESULT")
        print("=" * 80)
        if event.message is not None:
            print(f"\n{event.message.text}\n")
        print("=" * 80)

print("✓ Callback functions defined")

✓ Callback functions defined


## 🧪 Step 3.3: Build and Execute Magentic Workflow
---
Build the Magentic workflow with streaming callbacks and execute a complex task.

In [13]:
async def run_magentic_workflow():
    """Execute Magentic orchestration workflow."""
    
    # Complex task requiring both research and coding
    TASK = """
            Compare the total 24-hour operating cost and efficiency of two language model setups:
            DeepSeek-7B deployed on an NVIDIA H100 GPU instance (self-hosted)
            GPT-4.1-mini accessed through the Azure OpenAI Service
            Include the following points:
                24-hour cost estimation – instance hourly rate vs. API token cost
                Performance – average response time and throughput for 1K-token prompts
                Operational factors – maintenance effort and scalability
                Recommendation – which option offers better cost-to-value for continuous workloads
                Save Results:
                - saved to results/cost_summary.csv 
                - plot and save results/tokens_per_dollar.png
            Present the comparison in a simple table.
    """
    
    print("=" * 80)
    print("CASE 3: MAGENTIC ORCHESTRATION PATTERN - Intelligent Agent Coordination")
    print("=" * 80)
    print(f"\n📋 Complex Task: {TASK[:150]}...\n")
    
    # Reset streaming state
    global last_stream_agent_id, stream_line_open
    last_stream_agent_id = None
    stream_line_open = False
    
    # Build Magentic workflow
    print("🔧 Building Magentic workflow...\n")
    workflow = (
        MagenticBuilder()
        .participants(researcher=magentic_researcher, coder=magentic_coder)
        .on_event(on_magentic_event, mode=MagenticCallbackMode.STREAMING)
        .with_standard_manager(
            chat_client=AzureOpenAIChatClient(),
            max_round_count=10,
            max_stall_count=2,
            max_reset_count=2
        )
        .build()
    )
    
    print("🚀 Starting Magentic orchestration...\n")
    print("=" * 80 + "\n")
    
    # Execute workflow with streaming
    try:
        output: str | None = None
        async for event in workflow.run_stream(TASK):
            if isinstance(event, WorkflowOutputEvent):
                output = str(event.data)
        
        print("\n" + "=" * 80)
        print("MAGENTIC WORKFLOW COMPLETED")
        print("=" * 80)
        
        if output:
            print(f"\n📄 Final Output Available (length: {len(output)} characters)")
        else:
            print("\n⚠️ No final output received")
            
    except Exception as e:
        print(f"\n❌ Workflow execution failed: {e}")
        import traceback
        traceback.print_exc()

# Run Magentic workflow
await run_magentic_workflow()

[2025-10-14 09:08:27 - /anaconda/envs/venv_agentlab/lib/python3.12/site-packages/agent_framework/_workflows/_validation.py:522 - WARNING] Cycle detected in the workflow graph involving: agent_coder -> agent_researcher -> magentic_orchestrator -> agent_coder. Ensure termination or iteration limits exist.


CASE 3: MAGENTIC ORCHESTRATION PATTERN - Intelligent Agent Coordination

📋 Complex Task: 
            Compare the total 24-hour operating cost and efficiency of two language model setups:
            DeepSeek-7B deployed on an NVIDIA H100 ...

🔧 Building Magentic workflow...

🚀 Starting Magentic orchestration...



🎯 [ORCHESTRATOR: user_task]
────────────────────────────────────────────────────────────

            Compare the total 24-hour operating cost and efficiency of two language model setups:
            DeepSeek-7B deployed on an NVIDIA H100 GPU instance (self-hosted)
            GPT-4.1-mini accessed through the Azure OpenAI Service
            Include the following points:
                24-hour cost estimation – instance hourly rate vs. API token cost
                Performance – average response time and throughput for 1K-token prompts
                Operational factors – maintenance effort and scalability
                Recommendation – which option offers better cost-


🎯 [ORCHESTRATOR: task_ledger]
────────────────────────────────────────────────────────────

We are working to address the following user request:


            Compare the total 24-hour operating cost and efficiency of two language model setups:
            DeepSeek-7B deployed on an NVIDIA H100 GPU instance (self-hosted)
            GPT-4.1-mini accessed through the Azure OpenAI Service
            Include the following points:
                24-hour cost estimation – instance hourly rate vs. API token cost
                Performance – average response time and throughput for 1K-token prompts
                Operational factors – maintenance effort and scalability
                Recommendation – which option offers better cost-to-value for continuous workloads
                Save Results:
                - saved to results/cost_summary.csv 
                - plot and save results/tokens_per_dollar.png
            Present the comparison in a simple table.
    


To answer this req

### 📊 Magentic Orchestration Pattern Summary

**Advantages:**
- ✅ Intelligent planning and coordination
- ✅ Dynamic agent selection based on task
- ✅ Handles complex multi-step workflows
- ✅ Adaptive execution with progress monitoring
- ✅ Natural task decomposition

**Considerations:**
- ⚠️ More complex to set up
- ⚠️ Requires capable manager model
- ⚠️ Higher token consumption for planning
- ⚠️ May have longer execution time for simple tasks

**Best Use Cases:**
- Complex research projects
- Multi-step analysis workflows
- Tasks requiring different expertise
- Adaptive problem-solving
- Production-grade agent systems

# Comparison of Multi-Agent Patterns
---

## 📊 Pattern Comparison Matrix

| Aspect | Group Chat | Concurrent | Magentic |
|--------|-----------|------------|----------|
| **Execution** | Sequential | Parallel | Orchestrated |
| **Agent Interaction** | Turn-based dialogue | Independent work | Coordinated by manager |
| **Speed** | Slowest | Fastest | Medium |
| **Complexity** | Low | Low | High |
| **Planning** | None | None | Built-in |
| **Best For** | Refinement workflows | Independent analysis | Complex multi-step tasks |
| **Token Usage** | Medium | Low-Medium | High |
| **Observability** | High | Medium | Very High |

## 🎯 Decision Guide: When to Use Each Pattern

### Use **Group Chat** when:
- ✅ You need iterative refinement
- ✅ Agents should respond to each other's output
- ✅ Sequential review/approval is required
- ✅ Conversation flow is important
- ❌ Don't use for: Independent parallel work

### Use **Concurrent Execution** when:
- ✅ You need multiple independent perspectives
- ✅ Speed is critical
- ✅ Agents don't need to communicate
- ✅ You want diverse viewpoints
- ❌ Don't use for: Tasks requiring agent coordination

### Use **Magentic Orchestration** when:
- ✅ Task is complex with multiple steps
- ✅ Different agents have different capabilities
- ✅ You need intelligent task decomposition
- ✅ Adaptive execution is required
- ❌ Don't use for: Simple single-agent tasks

## 💡 Hybrid Approaches

You can also combine patterns:
- **Concurrent + Magentic**: Manager orchestrates groups of concurrent agents
- **Group Chat + Concurrent**: Run multiple group chats in parallel
- **Magentic → Group Chat**: Manager delegates to a group chat for refinement

# Key Takeaways
---

## 🎓 What You Learned

1. **Group Chat Pattern**
   - Sequential turn-based communication
   - Custom termination strategies
   - Perfect for collaborative refinement

2. **Concurrent Execution Pattern**
   - Parallel agent execution
   - Fast multi-perspective analysis
   - Simple fan-out/fan-in workflow

3. **Magentic Orchestration Pattern**
   - Intelligent task planning
   - Dynamic agent coordination
   - Complex multi-step workflows

## 🚀 Next Steps

- **Experiment with different agent instructions**: Try various persona combinations
- **Add more agents**: Scale up to 5-10 specialized agents
- **Integrate with real tools**: Connect to databases, APIs, file systems
- **Add human-in-the-loop**: Implement approval workflows
- **Monitor with observability**: Add Azure AI tracing and logging
- **Production deployment**: Use Azure AI Foundry for managed hosting

## 📚 Additional Resources

- [Microsoft Agent Framework Docs](https://learn.microsoft.com/en-us/agent-framework/)
- [Agent Framework Samples](https://github.com/microsoft/agent-framework/tree/main/python/samples)
- [Multi-Agent Design Patterns](https://learn.microsoft.com/en-us/agent-framework/patterns)
- [Azure AI Foundry](https://ai.azure.com/)